# Cox PH model

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

import pickle

from weighted_concordance import weighted_concordance

In [3]:
# pd.set_option('display.max_columns', None,'display.max_rows',20)

In [4]:
# importing module
import sys
  
# appending a path
sys.path.append('C:/Users/thseale/Documents/Python Packages')

from lifelines import CoxPHFitter
from lifelines.utils import datetimes_to_durations, k_fold_cross_validation, concordance_index

In [5]:
# importing data
X_tr = pd.read_csv("D:/compiled_data/X_tr.csv").set_index("csid")

y = pd.read_csv("D:/compiled_data/y.csv").set_index("csid")

In [6]:
X_tr

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,mother_still_alive_2,father_still_alive_0,father_still_alive_1,father_still_alive_2,weight_changed_0,weight_changed_1,weight_changed_2,hep_b_0.0,hep_b_1.0,hep_b_2.0
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1799093,0.122746,0.811057,0.592906,1.728535,-0.410683,-0.056127,1.780844,-1.604290,-0.884395,-0.449261,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2022-00229-4744170,-1.250938,-0.168420,-0.422900,0.301515,-0.402491,-0.056127,-0.277026,-1.096393,-1.183793,-0.787761,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2022-00229-1082517,-0.727339,0.040177,0.259928,-0.802161,0.560934,-0.056127,0.472728,0.249563,0.161982,-0.686003,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2022-00229-3397875,1.103472,0.999186,-0.013410,0.773113,1.712047,-0.056127,0.331545,1.448354,1.390868,0.250701,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2022-00229-4372175,1.805687,1.875552,1.842670,1.767708,2.068840,-0.056127,2.404542,1.382637,1.962589,3.259559,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-5565114,0.436560,-0.581341,-0.103120,0.316606,-0.240342,-0.056127,-0.783666,-0.087424,0.314348,0.772009,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2022-00229-4902412,-1.082104,-0.489840,0.057002,-0.483336,-0.998423,-0.056127,0.273152,-1.188568,-1.268748,-0.039717,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2022-00229-4823882,2.495089,0.353427,1.444274,0.410068,1.829385,-0.056127,0.517444,0.972800,1.380486,0.211005,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [7]:
y

,ep_CKB0020_combined_ep,ep_CKB0020_combined_datedeveloped,study_date,subcohort
csid,,,,
2022-00229-1799093,1,2009-07-23T00:00:00Z,2006-04-17T10:08:09Z,False
2022-00229-4744170,1,2011-05-13T00:00:00Z,2006-09-25T15:24:06Z,False
2022-00229-1082517,1,2010-07-14T00:00:00Z,2004-12-16T11:22:31Z,False
2022-00229-3397875,1,2013-06-29T00:00:00Z,2006-07-25T08:54:40Z,False
2022-00229-4372175,1,2013-06-03T00:00:00Z,2007-05-09T09:58:21Z,False
...,...,...,...,...
2022-00229-5565114,0,2018-12-31T00:00:00Z,2006-09-09T09:46:34Z,True
2022-00229-4902412,0,2016-01-02T00:00:00Z,2006-07-24T09:08:24Z,True
2022-00229-4823882,0,2007-09-29T00:00:00Z,2005-04-16T09:56:21Z,True


In [8]:
def date_time_conversion(date):
    return(datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ"))

In [9]:
# Coverting the times into datetime format
y_tr = y.assign(
    event = y.ep_CKB0020_combined_ep,
    start_time = y.study_date.map(date_time_conversion),
    event_time = y.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y.columns)
y_tr["duration"] = datetimes_to_durations(y_tr["start_time"],y_tr["event_time"])[0]
y_tr["subcohort"] = y["subcohort"]
y_tr = y_tr.drop(columns = ["start_time","event_time"])
y_tr

,event,duration,subcohort
csid,,,
2022-00229-1799093,1,1192.0,False
2022-00229-4744170,1,1690.0,False
2022-00229-1082517,1,2035.0,False
2022-00229-3397875,1,2530.0,False
2022-00229-4372175,1,2216.0,False
...,...,...,...
2022-00229-5565114,0,4495.0,True
2022-00229-4902412,0,3448.0,True
2022-00229-4823882,0,895.0,True


## Fitting Model

### Self-Prentice Cox PH

In [10]:
from sklearn.decomposition import PCA
from lifelines.utils import k_fold_cross_validation, concordance_index
from sklearn.model_selection import KFold

In [11]:
cox_selfprentice_penalizer_scores = pd.DataFrame({
    'penalizer' : [],
    'l1_ratio': [],
    'score': []
})
cox_selfprentice_penalizer_scores

,penalizer,l1_ratio,score


In [ ]:

for i in range(6,11):
    for j in range(0,3):
        
        print('penalizer =', i/5, 'l1_ratio =', j/3)
    
        kf = KFold(n_splits = 15)
        kf.get_n_splits(X_tr)

        scores = []

        for train_index, test_index in kf.split(X_tr):
            X_tr_train, X_tr_test = X_tr.iloc[train_index], X_tr.iloc[test_index]
            y_tr_train, y_tr_test = y_tr.iloc[train_index], y_tr.iloc[test_index]

            # creating a case-subcohort dataframe
            case_subcohort_selfprentice_df = X_tr_train.join(y_tr_train)

            case_subcohort_selfprentice_df["weight"] = np.where(case_subcohort_selfprentice_df["subcohort"],1,0.001)

            case_subcohort_selfprentice_df = case_subcohort_selfprentice_df.drop(columns = "subcohort")

            case_subcohort_selfprentice_df

            cph = CoxPHFitter(penalizer = i/5, l1_ratio = j/3)
            
            try:
                cph.fit(case_subcohort_selfprentice_df, duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

                event_times = y_tr_test["duration"]
                event_observed = y_tr_test["event"]

                test_preds = cph.predict_partial_hazard(X_tr_test)
                scores.append(weighted_concordance(event_times,test_preds, event_observed,0.0011604684001529089))
            except:
                pass
            
        cox_selfprentice_penalizer_scores = cox_selfprentice_penalizer_scores.append({
            'penalizer' : i/5,
            'l1_ratio': j/3,
            'score': np.mean(scores)
        }, ignore_index = True)

penalizer = 1.2 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.2 l1_ratio = 0.3333333333333333


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.2 l1_ratio = 0.6666666666666666


In [ ]:
cox_selfprentice_penalizer_scores

In general, increasing the L1 ratio appears to decrease predictive performance. Let us search from 1.2-1.8 for penalisation find the best performance:


In [24]:
for i in range(12,18):
    for j in range(0,1):
        
        print('penalizer =', i/10, 'l1_ratio =', j/3)
    
        kf = KFold(n_splits = 15)
        kf.get_n_splits(X_tr)

        scores = []

        for train_index, test_index in kf.split(X_tr):
            X_tr_train, X_tr_test = X_tr.iloc[train_index], X_tr.iloc[test_index]
            y_tr_train, y_tr_test = y_tr.iloc[train_index], y_tr.iloc[test_index]

            # creating a case-subcohort dataframe
            case_subcohort_selfprentice_df = X_tr_train.join(y_tr_train)

            case_subcohort_selfprentice_df["weight"] = np.where(case_subcohort_selfprentice_df["subcohort"],1,0.001)

            case_subcohort_selfprentice_df = case_subcohort_selfprentice_df.drop(columns = "subcohort")

            case_subcohort_selfprentice_df

            cph = CoxPHFitter(penalizer = i/10, l1_ratio = j/3)
            
            try:
                cph.fit(case_subcohort_selfprentice_df, duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

                event_times = y_tr_test["duration"]
                event_observed = y_tr_test["event"]

                test_preds = cph.predict_partial_hazard(X_tr_test)
                scores.append(concordance_index(event_times, -1*test_preds, event_observed))
            except:
                pass
            
        cox_selfprentice_penalizer_scores = cox_selfprentice_penalizer_scores.append({
            'penalizer' : i/10,
            'l1_ratio': j/3,
            'score': np.mean(scores)
        }, ignore_index = True)

penalizer = 1.2 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.3 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.4 l1_ratio = 0.0


...

C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.6 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.7 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

In [30]:
cox_selfprentice_penalizer_scores.dropna().query("l1_ratio == 0").sort_values("penalizer")

,penalizer,l1_ratio,score
15,1.2,0.0,0.572268
30,1.2,0.0,0.572268
31,1.3,0.0,0.572267
18,1.4,0.0,0.572267
32,1.4,0.0,0.572267
33,1.5,0.0,0.572303
21,1.6,0.0,0.572303
34,1.6,0.0,0.572303
35,1.7,0.0,0.572173
24,1.8,0.0,0.572081


1.5-1.6 seems to be the best penaliser.

Fitting final model:

In [32]:
# creating a case-subcohort dataframe
case_subcohort_selfprentice_df = X_tr.join(y_tr)

case_subcohort_selfprentice_df["weight"] = np.where(case_subcohort_selfprentice_df["subcohort"],1,0.001)

case_subcohort_selfprentice_df = case_subcohort_selfprentice_df.drop(columns = "subcohort")

In [35]:
cph3 = CoxPHFitter(penalizer = 1.55)
cph3.fit(case_subcohort_selfprentice_df, duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

<lifelines.CoxPHFitter: fitted with 595.595 total observations, 593 right-censored observations>

In [37]:
pickle.dump(cph3, open('penalized_selfprentice_cox.pkl', 'wb'))